<a href="https://colab.research.google.com/github/Masum06/Text2App/blob/master/Text2App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [1]:
!apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (738 kB/s)
Selecting previously unselected package tree.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


## Imports

In [0]:
import re
import random
from random import shuffle
import xml.dom.minidom
import pandas as pd

# **Neural Machine Translation model for Text to SAR**

In [0]:
df = pd.read_csv("text2app_data_1000.csv")

In [8]:
df

,input_text,target_sar
0,create an app with button1 . when button1 pre...,<start_vis> <button> <accelerometer> <video> <...
1,create an app with button1 and an accelerome...,<start_vis> <button> <accelerometer> <video> <...
2,create an app with button1 and an accelerome...,<start_vis> <button> <accelerometer> <video> <...
3,create an app with button1 button2 . when but...,<start_vis> <button> <button> <accelerometer> ...
4,create an app with button1 button2 and an ac...,<start_vis> <button> <button> <accelerometer> ...
...,...,...
1995,create an app with button1 button2 button3 a...,<start_vis> <button> <button> <button> <accele...
1996,create an app with button1 button2 button3 a...,<start_vis> <button> <button> <button> <accele...
1997,create an app with button1 button2 button3 . ...,<start_vis> <button> <button> <button> <accele...
1998,create an app with an accelerometer sensor . ...,<start_vis> <accelerometer> <video> <camera> ...


In [41]:
fp_input = open("src_test.txt", 'w+')
fp_input.write('\n'.join(list(df['input_text'].values)))
fp_input.close

<function TextIOWrapper.close>

In [42]:
fp_output = open("tgt_test.txt", 'w+')
fp_output.write('\n'.join(list(df['target_sar'].values)))
fp_output.close

<function TextIOWrapper.close>

Preprocessing

### Training with OpenNMT

In [2]:
!git clone https://github.com/OpenNMT/OpenNMT-py

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15223 (delta 1), reused 4 (delta 0), pack-reused 15211
Receiving objects: 100% (15223/15223), 146.37 MiB | 12.20 MiB/s, done.
Resolving deltas: 100% (10938/10938), done.


In [3]:
!pip install OpenNMT-py

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 2.2MB 37.5MB/s 
     |████████████████████████████████| 40kB 5.9MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.0-cp36-none-any.whl size=17874 sha256=1eb3fff8db1a2e4cc6c4a29cb9d03afd8162ed1854887054c0c4a2cb01094437
  Stored in directory: /root/.cache/pip/wheels/0d/3a/3f/d9c3d44d4e15f125c92bd64b611106937511e20ecc34bb5229
Successfully built configargparse
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


http://opennmt.net/OpenNMT-py/quickstart.html

http://opennmt.net/OpenNMT-py/options/train.html

In [1]:
!python OpenNMT-py/preprocess.py -train_src data/src_train.txt -train_tgt data/tgt_train.txt \
-valid_src data/src_val.txt -valid_tgt data/tgt_val.txt -save_data data/data

[2019-12-31 01:26:03,925 INFO] Extracting features...
[2019-12-31 01:26:03,925 INFO]  * number of source features: 0.
[2019-12-31 01:26:03,925 INFO]  * number of target features: 0.
[2019-12-31 01:26:03,925 INFO] Building `Fields` object...
[2019-12-31 01:26:03,926 INFO] Building & saving training data...
[2019-12-31 01:26:03,941 INFO] Building shard 0.
[2019-12-31 01:26:04,009 INFO]  * saving 0th train data shard to data/data.train.0.pt.
[2019-12-31 01:26:04,135 INFO]  * tgt vocab size: 40.
[2019-12-31 01:26:04,135 INFO]  * src vocab size: 36.
[2019-12-31 01:26:04,136 INFO] Building & saving validation data...
[2019-12-31 01:26:04,144 INFO] Building shard 0.
[2019-12-31 01:26:04,155 INFO]  * saving 0th valid data shard to data/data.valid.0.pt.


In [0]:
!export CUDA_VISIBLE_DEVICES

In [6]:
!python OpenNMT-py/train.py \
-global_attention mlp \
-world_size 1 -gpu_ranks 0 \
-train_from drive/"My Drive"/text2app_models/Text2App_step_20000.pt \
-train_steps 30000 -save_checkpoint_steps 500 \
-data data/data \
-save_model drive/"My Drive"/text2app_models/Text2App

[2019-12-31 03:08:21,367 INFO] Loading checkpoint from drive/My Drive/text2app_models/Text2App_step_20000.pt
[2019-12-31 03:08:21,448 INFO] Loading vocab from checkpoint at drive/My Drive/text2app_models/Text2App_step_20000.pt.
[2019-12-31 03:08:21,448 INFO]  * src vocab size = 36
[2019-12-31 03:08:21,449 INFO]  * tgt vocab size = 40
[2019-12-31 03:08:21,449 INFO] Building model...
[2019-12-31 03:08:24,792 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(36, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(40, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False

In [0]:
!cp models/* drive/"My Drive"/text2app_models/

In [8]:
!python OpenNMT-py/translate.py -model drive/"My Drive"/text2app_models/Text2App_step_30000.pt -src \
data/src_test.txt -output data/pred.txt -replace_unk -verbose -beam_size 1

[2019-12-31 03:31:25,982 INFO] Translating shard 0.

SENT 1: ['create', 'an', 'app', 'with', 'button1', 'button2', '.', 'when', 'button1', 'pressed', ',', 'play', 'video', 'breakingbad.mp4', '.', 'when', 'button2', 'pressed', ',', 'stop', 'player', '.']
PRED 1: <start_vis> <button> <button> <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> <button1> <play_video> breakingbad.mp4 </play_video> </button> <new_line> <button2> <stop_play> </button> <new_line> <end_code>
PRED SCORE: 0.0000

SENT 2: ['create', 'an', 'app', 'with', 'an', 'accelerometer', 'sensor', '.', 'when', 'phone', 'is', 'shaken', ',', 'open', 'the', 'camera', '.']
PRED 2: <start_vis> <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> <shaken> <open_camera> </shaken> <new_line> <end_code>
PRED SCORE: 0.0000

SENT 3: ['create', 'an', 'app', 'with', 'button1', 'button2', '.', 'when', 'button1', 'pressed', ',', 'open', 'the', 'camera', '.', 'when', 'button2', 'pressed',

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files

files.download('model/Text2App_500')

In [0]:
!mkdir drive/"My Drive"/text2app_models

### Training with Glove pretrained word embedding

In [0]:
!mkdir data

In [62]:
!python OpenNMT-py/preprocess.py -train_src src_train.txt -train_tgt tgt_train.txt \
-valid_src src_val.txt -valid_tgt tgt_val.txt -save_data data/data

[2019-12-30 18:32:53,659 INFO] Extracting features...
[2019-12-30 18:32:53,660 INFO]  * number of source features: 0.
[2019-12-30 18:32:53,660 INFO]  * number of target features: 0.
[2019-12-30 18:32:53,660 INFO] Building `Fields` object...
[2019-12-30 18:32:53,660 INFO] Building & saving training data...
[2019-12-30 18:32:53,673 INFO] Building shard 0.
[2019-12-30 18:32:53,728 INFO]  * saving 0th train data shard to data/data.train.0.pt.
[2019-12-30 18:32:53,868 INFO]  * tgt vocab size: 40.
[2019-12-30 18:32:53,868 INFO]  * src vocab size: 36.
[2019-12-30 18:32:53,869 INFO] Building & saving validation data...
[2019-12-30 18:32:53,876 INFO] Building shard 0.
[2019-12-30 18:32:53,885 INFO]  * saving 0th valid data shard to data/data.valid.0.pt.


In [47]:
!mkdir glove
!mkdir embedding

mkdir: cannot create directory ‘glove’: File exists


In [48]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-12-30 18:17:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-30 18:17:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-30 18:17:00--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [57]:
!unzip glove.6B.zip.1 -d glove

Archive:  glove.6B.zip.1
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  


In [64]:
!python OpenNMT-py/tools/embeddings_to_torch.py -emb_file_both "glove/glove.6B.100d.txt" \
                                 -dict_file "data/data.vocab.pt" \
                                 -output_file "embedding/embeddings" 

[2019-12-30 18:37:53,634 INFO] From: data/data.vocab.pt
[2019-12-30 18:37:53,634 INFO] 	* source vocab: 36 words
[2019-12-30 18:37:53,634 INFO] 	* target vocab: 40 words
[2019-12-30 18:37:53,634 INFO] Reading encoder and decoder embeddings from glove/glove.6B.100d.txt
[2019-12-30 18:37:55,877 INFO] 	Found 400000 total vectors in file
[2019-12-30 18:37:55,877 INFO] After filtering to vectors in vocab:
[2019-12-30 18:37:55,877 INFO] 	* enc: 22 match, 14 missing, (61.11%)
[2019-12-30 18:37:55,877 INFO] 	* dec: 0 match, 40 missing, (0.00%)
[2019-12-30 18:37:55,877 INFO] 
Saving embedding as:
	* enc: embedding/embeddings.enc.pt
	* dec: embedding/embeddings.dec.pt
[2019-12-30 18:37:55,879 INFO] 
Done.


In [72]:
!python OpenNMT-py/train.py -save_model model/Text2app -train_steps 2500 -batch_size 50 -layers 2 -rnn_size 200 -word_vec_size 100 -pre_word_vecs_enc "embedding/embeddings.enc.pt" -pre_word_vecs_dec "embedding/embeddings.dec.pt" -data OpenNMT-py/data/demo

You have a CUDA device, should run with -gpu_ranks
[2019-12-30 18:41:12,239 INFO]  * src vocab size = 24997
[2019-12-30 18:41:12,239 INFO]  * tgt vocab size = 35820
[2019-12-30 18:41:12,239 INFO] Building model...
Traceback (most recent call last):
  File "OpenNMT-py/train.py", line 6, in <module>
    main()
  File "/content/OpenNMT-py/onmt/bin/train.py", line 204, in main
    train(opt)
  File "/content/OpenNMT-py/onmt/bin/train.py", line 90, in train
    single_main(opt, -1)
  File "/content/OpenNMT-py/onmt/train_single.py", line 84, in main
    model = build_model(model_opt, opt, fields, checkpoint)
  File "/content/OpenNMT-py/onmt/model_builder.py", line 240, in build_model
    model = build_base_model(model_opt, fields, use_gpu(opt), checkpoint)
  File "/content/OpenNMT-py/onmt/model_builder.py", line 226, in build_base_model
    model_opt.pre_word_vecs_enc)
  File "/content/OpenNMT-py/onmt/modules/embeddings.py", line 258, in load_pretrained_vectors
    self.word_lut.weight.data.

In [45]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

Hello Google Drive!

Inference

In [0]:
!python OpenNMT-py/translate.py -model OpenNMT-py/data/model/demo-model_YOUR_MODEL.pt -src \
src-test.txt -output pred.txt -replace_unk -verbose -beam_size 1

In [0]:
sed -i "s/@@ //g"  OpenNMT-py/data/pred.txt

# **SAR to App**
SAR = Simplified App Representation

## Input

In [0]:
project_name = "GeneratedProject1"

In [0]:
user_name = "youngladesh"

In [0]:
SAR = "<start_vis> <button> <button> <button> <button> <accelerometer> <video> <camera> <text2speech> <player> <end_vis> \
<start_code> <button1> <play> panini.mp3 </play> </button> <new_line> <shaken> <speak> <text> something like this </text> \
</speak> </shaken> <new_line> <button2> <open_camera>  </button> <new_line> <button3> <play_video> starwars.mp4 </play_video> \
</button> <new_line> <button4> <stop_play> </button> <new_line> <end_code>"
tokens = SAR.split()

## Logic Components

In [0]:
dic = {
	"<start_code>" : """<xml xmlns="http://www.w3.org/1999/xhtml">""",

	"<end_code>" :   """<yacodeblocks ya-version="196" language-version="27"></yacodeblocks>
	</xml>""",

	"<button1>" : """<block type="component_event" id="5Fnc2eKTqXnvQEkuEb,`" x="-717" y="-541">
	    <mutation component_type="Button" is_generic="false" instance_name="Button1" event_name="Click"></mutation>
	    <field name="COMPONENT_SELECTOR">Button1</field>
	    <statement name="DO">""",

	"<button2>" : """<block type="component_event" id="L.k%OOfugWWKbB|X=PTB" x="-208" y="-503">
    <mutation component_type="Button" is_generic="false" instance_name="Button2" event_name="Click"></mutation>
    <field name="COMPONENT_SELECTOR">Button2</field>
    <statement name="DO">""",

    "<button3>" : """<block type="component_event" id="*H.?`=wW)@O-=dzx80i%" x="-227" y="-369">
    <mutation component_type="Button" is_generic="false" instance_name="Button3" event_name="Click"></mutation>
    <field name="COMPONENT_SELECTOR">Button3</field>
    <statement name="DO">""",

    "<button4>" : """<block type="component_event" id="V`+dX.LLP6k;~`}Y_Sy+" x="-214" y="-225">
    <mutation component_type="Button" is_generic="false" instance_name="Button4" event_name="Click"></mutation>
    <field name="COMPONENT_SELECTOR">Button4</field>
    <statement name="DO">
""",

	"</button>" : """</statement>
  </block>""",

	"<play>" : """<block type="component_set_get" id="M`ORo822oPPT_NZ5UQl8">
        <mutation component_type="Player" set_or_get="set" property_name="Source" is_generic="false" instance_name="Player1"></mutation>
        <field name="COMPONENT_SELECTOR">Player1</field>

        <field name="PROP">Source</field>

        <value name="VALUE">
          <block type="text" id="Hv*@f(.v|qmJFJ5cpEY@">
            <field name="TEXT">""",

    "</play>" : """</field>	
          </block>	
        </value>	
        <next>	
          <block type="component_method" id="*|w:Ln#[K;pY=W:C0Tm2">	
            <mutation component_type="Player" method_name="Start" is_generic="false" instance_name="Player1"></mutation>	
            <field name="COMPONENT_SELECTOR">Player1</field>	
          </block>	
        </next>	
      </block>	""",

    "<new_line>" : "\n",

    "<shaken>" : """<block type="component_event" id="(mbR6!,f3BycMC6fV#vp" x="-740" y="-367">
    <mutation component_type="AccelerometerSensor" is_generic="false" instance_name="AccelerometerSensor1" event_name="Shaking"></mutation>
    <field name="COMPONENT_SELECTOR">AccelerometerSensor1</field>
    <statement name="DO">""",

   	"</shaken>" : """</statement>
  </block>""",

  	"<speak>" : """<block type="component_method" id="uk1$YtD.~FFzEI{,^jo?">
        <mutation component_type="TextToSpeech" method_name="Speak" is_generic="false" instance_name="TextToSpeech1"></mutation>
        <field name="COMPONENT_SELECTOR">TextToSpeech1</field>
        <value name="ARG0">
          <block type="text" id="Pd3e?.zfPi`8U[y:Lq[I">
            <field name="TEXT">""",

    "</speak>" : """</field>
          </block>
        </value>
      </block>""",

    "<open_camera>" : """<block type="component_method" id="w~=BBIJa$W%0rs}t%Gr0">
        <mutation component_type="Camera" method_name="TakePicture" is_generic="false" instance_name="Camera1"></mutation>
        <field name="COMPONENT_SELECTOR">Camera1</field>
        </block>""",

    '<play_video>' : """<block type="component_set_get" id="~@geDXo-K5C9N[!Q}9/}">
        <mutation component_type="VideoPlayer" set_or_get="set" property_name="Source" is_generic="false" instance_name="VideoPlayer1"></mutation>
        <field name="COMPONENT_SELECTOR">VideoPlayer1</field>
        <field name="PROP">Source</field>
        <value name="VALUE">
          <block type="text" id="c~qz-Tkkz9E}()oRV,`i">
            <field name="TEXT">""",

    "</play_video>" : """</field>
          </block>
        </value>
        <next>
          <block type="component_method" id="OR/@4AKMZY7KRq?ACKxg">
            <mutation component_type="VideoPlayer" method_name="Start" is_generic="false" instance_name="VideoPlayer1"></mutation>
            <field name="COMPONENT_SELECTOR">VideoPlayer1</field>
          </block>
        </next>
      </block>""",


      "<stop_play>" : """<block type="component_method" id="|!X?VEIdC{:M92}$8Sev">
        <mutation component_type="Player" method_name="Stop" is_generic="false" instance_name="Player1"></mutation>
        <field name="COMPONENT_SELECTOR">Player1</field>
      </block>"""

}

In [0]:
dic

{'</button>': '</statement>\n  </block>',
 '</play>': '</field>\t\n          </block>\t\n        </value>\t\n        <next>\t\n          <block type="component_method" id="*|w:Ln#[K;pY=W:C0Tm2">\t\n            <mutation component_type="Player" method_name="Start" is_generic="false" instance_name="Player1"></mutation>\t\n            <field name="COMPONENT_SELECTOR">Player1</field>\t\n          </block>\t\n        </next>\t\n      </block>\t',
 '</play_video>': '</field>\n          </block>\n        </value>\n        <next>\n          <block type="component_method" id="OR/@4AKMZY7KRq?ACKxg">\n            <mutation component_type="VideoPlayer" method_name="Start" is_generic="false" instance_name="VideoPlayer1"></mutation>\n            <field name="COMPONENT_SELECTOR">VideoPlayer1</field>\n          </block>\n        </next>\n      </block>',
 '</shaken>': '</statement>\n  </block>',
 '</speak>': '</field>\n          </block>\n        </value>\n      </block>',
 '<button1>': '<block type="

In [0]:
tokens

['<start_vis>',
 '<button>',
 '<button>',
 '<button>',
 '<button>',
 '<accelerometer>',
 '<video>',
 '<camera>',
 '<text2speech>',
 '<player>',
 '<end_vis>',
 '<start_code>',
 '<button1>',
 '<play_video>',
 'starwars.mp4',
 '</play_video>',
 '</button>',
 '<new_line>',
 '<button2>',
 '<play>',
 'lightsaber.wav',
 '</play>',
 '</button>',
 '<new_line>',
 '<button3>',
 '<stop_play>',
 '</button>',
 '<new_line>',
 '<button4>',
 '<play_video>',
 'breakingbad.mp4',
 '</play_video>',
 '</button>',
 '<new_line>',
 '<shaken>',
 '<play_video>',
 'breakingbad.mp4',
 '</play_video>',
 '</shaken>',
 '<new_line>',
 '<end_code>']

In [0]:
vis_tokens = tokens[1:tokens.index("<end_vis>")]
code_tokens = tokens[tokens.index("<end_vis>")+1:]

In [0]:
vis_tokens

['<button>',
 '<button>',
 '<button>',
 '<button>',
 '<accelerometer>',
 '<video>',
 '<camera>',
 '<text2speech>',
 '<player>']

In [0]:
code_tokens

['<start_code>',
 '<button1>',
 '<play_video>',
 'starwars.mp4',
 '</play_video>',
 '</button>',
 '<new_line>',
 '<button2>',
 '<play>',
 'lightsaber.wav',
 '</play>',
 '</button>',
 '<new_line>',
 '<button3>',
 '<stop_play>',
 '</button>',
 '<new_line>',
 '<button4>',
 '<play_video>',
 'breakingbad.mp4',
 '</play_video>',
 '</button>',
 '<new_line>',
 '<shaken>',
 '<play_video>',
 'breakingbad.mp4',
 '</play_video>',
 '</shaken>',
 '<new_line>',
 '<end_code>']

In [0]:
i = 0
bky = ""
while i < len(code_tokens):
  #print(i)
  token = code_tokens[i]
  if token == '<text>':
    i+=1
    text = ""
    while True:
      token = code_tokens[i]
      if token == '</text>':
        break
      text += token+" "
      i+=1
    bky += text
  elif token == '<play>' or token == '<play_video>':
    bky += dic[token]
    i += 1
    token = code_tokens[i]
    bky += token
  else:
    bky += dic[token]
  i+=1

In [0]:
bky

'<xml xmlns="http://www.w3.org/1999/xhtml"><block type="component_event" id="5Fnc2eKTqXnvQEkuEb,`" x="-717" y="-541">\n\t    <mutation component_type="Button" is_generic="false" instance_name="Button1" event_name="Click"></mutation>\n\t    <field name="COMPONENT_SELECTOR">Button1</field>\n\t    <statement name="DO"><block type="component_set_get" id="~@geDXo-K5C9N[!Q}9/}">\n        <mutation component_type="VideoPlayer" set_or_get="set" property_name="Source" is_generic="false" instance_name="VideoPlayer1"></mutation>\n        <field name="COMPONENT_SELECTOR">VideoPlayer1</field>\n        <field name="PROP">Source</field>\n        <value name="VALUE">\n          <block type="text" id="c~qz-Tkkz9E}()oRV,`i">\n            <field name="TEXT">starwars.mp4</field>\n          </block>\n        </value>\n        <next>\n          <block type="component_method" id="OR/@4AKMZY7KRq?ACKxg">\n            <mutation component_type="VideoPlayer" method_name="Start" is_generic="false" instance_name="V

In [0]:
dom = xml.dom.minidom.parseString(bky) #xml.dom.minidom.parse(xml_fname) # or 
pretty_xml_as_string = dom.toprettyxml()
pretty_xml_as_string

'<?xml version="1.0" ?>\n<xml xmlns="http://www.w3.org/1999/xhtml">\n\t<block id="5Fnc2eKTqXnvQEkuEb,`" type="component_event" x="-717" y="-541">\n\t\t\n\t    \n\t\t<mutation component_type="Button" event_name="Click" instance_name="Button1" is_generic="false"/>\n\t\t\n\t    \n\t\t<field name="COMPONENT_SELECTOR">Button1</field>\n\t\t\n\t    \n\t\t<statement name="DO">\n\t\t\t<block id="~@geDXo-K5C9N[!Q}9/}" type="component_set_get">\n\t\t\t\t\n        \n\t\t\t\t<mutation component_type="VideoPlayer" instance_name="VideoPlayer1" is_generic="false" property_name="Source" set_or_get="set"/>\n\t\t\t\t\n        \n\t\t\t\t<field name="COMPONENT_SELECTOR">VideoPlayer1</field>\n\t\t\t\t\n        \n\t\t\t\t<field name="PROP">Source</field>\n\t\t\t\t\n        \n\t\t\t\t<value name="VALUE">\n\t\t\t\t\t\n          \n\t\t\t\t\t<block id="c~qz-Tkkz9E}()oRV,`i" type="text">\n\t\t\t\t\t\t\n            \n\t\t\t\t\t\t<field name="TEXT">starwars.mp4</field>\n\t\t\t\t\t\t\n          \n\t\t\t\t\t</block>\

In [0]:
print(pretty_xml_as_string)

<?xml version="1.0" ?>
<xml xmlns="http://www.w3.org/1999/xhtml">
	<block id="5Fnc2eKTqXnvQEkuEb,`" type="component_event" x="-717" y="-541">
		
	    
		<mutation component_type="Button" event_name="Click" instance_name="Button1" is_generic="false"/>
		
	    
		<field name="COMPONENT_SELECTOR">Button1</field>
		
	    
		<statement name="DO">
			<block id="~@geDXo-K5C9N[!Q}9/}" type="component_set_get">
				
        
				<mutation component_type="VideoPlayer" instance_name="VideoPlayer1" is_generic="false" property_name="Source" set_or_get="set"/>
				
        
				<field name="COMPONENT_SELECTOR">VideoPlayer1</field>
				
        
				<field name="PROP">Source</field>
				
        
				<value name="VALUE">
					
          
					<block id="c~qz-Tkkz9E}()oRV,`i" type="text">
						
            
						<field name="TEXT">starwars.mp4</field>
						
          
					</block>
					
        
				</value>
				
        
				<next>
					
          
					<block id="OR/@4AKMZY7KRq?ACKxg" type="comp

## Visual Componnets

In [0]:
component_dict = {
    "button_count": 0,
    "max_button_count":3,
    "buttons": [
      {"$Name":"Button1","$Type":"Button","$Version":"6","Text":"Text for Button1","Uuid":"-1018622447"},
      {"$Name":"Button2","$Type":"Button","$Version":"6","Text":"Text for Button2","Uuid":"962267658"},
      {"$Name":"Button3","$Type":"Button","$Version":"6","Text":"Text for Button3","Uuid":"1669738041"},
      {"$Name":"Button4","$Type":"Button","$Version":"6","Text":"Text for Button4","Uuid":"1669558870"}
    ],
    "<video>": {"$Name":"VideoPlayer1","$Type":"VideoPlayer","$Version":"6","Uuid":"-894931296"},
    "<accelerometer>": {"$Name":"AccelerometerSensor1","$Type":"AccelerometerSensor","$Version":"4","Uuid":"1490551342"},
    "<camera>": {"$Name":"Camera1","$Type":"Camera","$Version":"3","Uuid":"-569313149"},
    "<text2speech>": {"$Name":"TextToSpeech1","$Type":"TextToSpeech","$Version":"5","Uuid":"-1685610153"},
    "<player>": {"$Name":"Player1","$Type":"Player","$Version":"6","Uuid":"-382824747"}
}

In [0]:
scm = """#|
$JSON
{"authURL":["ai2.appinventor.mit.edu"],"YaVersion":"196","Source":"Form","Properties":{"$Name":"Screen1","$Type":"Form","$Version":"26","ActionBar":"True","AppName":"<|app_name|>","ShowListsAsJson":"False","Sizing":"Fixed","Theme":"AppTheme.Light.DarkActionBar","Title":"My App from Text","Uuid":"0",
"$Components": <|components|>
}}
|#"""

In [0]:
vis_components = []
for token in vis_tokens:
  if token == "<button>" and component_dict["button_count"]<=component_dict["max_button_count"]:
    component = component_dict["buttons"][component_dict["button_count"]]
    component_dict["button_count"] += 1
  else:
    component = component_dict[token]
  vis_components.append(component)

In [0]:
str(vis_components)

"[{'$Name': 'Button1', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button1', 'Uuid': '-1018622447'}, {'$Name': 'Button2', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button2', 'Uuid': '962267658'}, {'$Name': 'Button3', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button3', 'Uuid': '1669738041'}, {'$Name': 'Button4', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button4', 'Uuid': '1669558870'}, {'$Name': 'AccelerometerSensor1', '$Type': 'AccelerometerSensor', '$Version': '4', 'Uuid': '1490551342'}, {'$Name': 'VideoPlayer1', '$Type': 'VideoPlayer', '$Version': '6', 'Uuid': '-894931296'}, {'$Name': 'Camera1', '$Type': 'Camera', '$Version': '3', 'Uuid': '-569313149'}, {'$Name': 'TextToSpeech1', '$Type': 'TextToSpeech', '$Version': '5', 'Uuid': '-1685610153'}, {'$Name': 'Player1', '$Type': 'Player', '$Version': '6', 'Uuid': '-382824747'}]"

In [0]:
print(scm)

#|
$JSON
{"authURL":["ai2.appinventor.mit.edu"],"YaVersion":"196","Source":"Form","Properties":{"$Name":"Screen1","$Type":"Form","$Version":"26","ActionBar":"True","AppName":"<|app_name|>","ShowListsAsJson":"False","Sizing":"Fixed","Theme":"AppTheme.Light.DarkActionBar","Title":"My App from Text","Uuid":"0",
"$Components": <|components|>
}}
|#


In [0]:
vis_file = scm.replace("<|app_name|>", project_name).replace("<|components|>", str(vis_components))
print(vis_file)

#|
$JSON
{"authURL":["ai2.appinventor.mit.edu"],"YaVersion":"196","Source":"Form","Properties":{"$Name":"Screen1","$Type":"Form","$Version":"26","ActionBar":"True","AppName":"GeneratedProject1","ShowListsAsJson":"False","Sizing":"Fixed","Theme":"AppTheme.Light.DarkActionBar","Title":"My App from Text","Uuid":"0",
"$Components": [{'$Name': 'Button1', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button1', 'Uuid': '-1018622447'}, {'$Name': 'Button2', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button2', 'Uuid': '962267658'}, {'$Name': 'Button3', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button3', 'Uuid': '1669738041'}, {'$Name': 'Button4', '$Type': 'Button', '$Version': '6', 'Text': 'Text for Button4', 'Uuid': '1669558870'}, {'$Name': 'AccelerometerSensor1', '$Type': 'AccelerometerSensor', '$Version': '4', 'Uuid': '1490551342'}, {'$Name': 'VideoPlayer1', '$Type': 'VideoPlayer', '$Version': '6', 'Uuid': '-894931296'}, {'$Name': 'Camera1', '$Type': 'Camera', '

In [0]:
vis_file = vis_file.replace("\'", "\"")

## Project Properties

In [0]:
base_properties = """
main=appinventor.ai_{0}.{1}.Screen1
name=BlankProject
assets=../assets
source=../src
build=../build
versioncode=1
versionname=1.0
useslocation=False
aname={1}
sizing=Fixed
showlistsasjson=False
actionbar=True
theme=AppTheme.Light.DarkActionBar
color.primary=&HFF3F51B5
color.primary.dark=&HFF303F9F
color.accent=&HFFFF4081
"""

In [0]:
project_properties = base_properties.format(user_name, project_name)

In [0]:
print(project_properties)


main=appinventor.ai_youngladesh.GeneratedProject1.Screen1
name=BlankProject
assets=../assets
source=../src
build=../build
versioncode=1
versionname=1.0
useslocation=False
aname=GeneratedProject1
sizing=Fixed
showlistsasjson=False
actionbar=True
theme=AppTheme.Light.DarkActionBar
color.primary=&HFF3F51B5
color.primary.dark=&HFF303F9F
color.accent=&HFFFF4081



## Media files

In [0]:
%%bash
git clone https://github.com/jeroenmeulenaar/python3-mega.git python3mega
cd python3mega
pip install -r requirements.txt

fatal: destination path 'python3mega' already exists and is not an empty directory.


In [0]:
import os
os.chdir('python3mega')
from mega import Mega
m = Mega.from_ephemeral()
os.chdir('..')

In [0]:
url = 'https://mega.nz/#!tA4GnAZC!H79PelVSfG0l6RzNdgB7EPOGzaJXzmrRKmhUVyGtlaU'

In [0]:
m.download_from_url(url)

In [0]:
!unzip -o text2app_media.zip

Archive:  text2app_media.zip
  inflating: text2app_media/bird.wav  
  inflating: text2app_media/breakingbad.mp4  
  inflating: text2app_media/car.wav  
  inflating: text2app_media/lightsaber.wav  
  inflating: text2app_media/panini.mp3  
  inflating: text2app_media/spiderverse.mp4  
  inflating: text2app_media/starwars.mp4  


In [0]:
!ls text2app_media

bird.wav	 car.wav	 panini.mp3	  starwars.mp4
breakingbad.mp4  lightsaber.wav  spiderverse.mp4


## Reverse Engineering App Inventor Export file

In [0]:
# .
# ├── assets
# │   ├── panini.mp3
# │   └── starwars.mp4
# ├── src
# │   └── appinventor
# │       └── ai_youngladesh
# │           └── BlankProject
# │               ├── Screen1.bky
# │               └── Screen1.scm
# └── youngandroidproject
#     └── project.properties

In [0]:
if os.path.exists('myapp'):
  !rm -r myapp

In [0]:
import os
os.makedirs('myapp/src/appinventor/ai_{0}/{1}'.format(user_name, project_name))
os.makedirs('myapp/youngandroidproject')

In [0]:
# Comment while debugging
os.makedirs('myapp/assets')
!cp text2app_media/* myapp/assets/

In [0]:
bky_file = open('myapp/src/appinventor/ai_{0}/{1}/Screen1.bky'.format(user_name, project_name), 'w+')
scm_file = open('myapp/src/appinventor/ai_{0}/{1}/Screen1.scm'.format(user_name, project_name), 'w+')
properties = open('myapp/youngandroidproject/project.properties', 'w+')

In [0]:
bky_file.write(pretty_xml_as_string)
scm_file.write(vis_file)
properties.write(project_properties)

360

In [0]:
bky_file.close()
scm_file.close()
properties.close()

In [0]:
%%bash
cd myapp
zip -r myapp.zip *
mv myapp.zip ..

  adding: src/ (stored 0%)
  adding: src/appinventor/ (stored 0%)
  adding: src/appinventor/ai_youngladesh/ (stored 0%)
  adding: src/appinventor/ai_youngladesh/GeneratedProject1/ (stored 0%)
  adding: src/appinventor/ai_youngladesh/GeneratedProject1/Screen1.scm (deflated 62%)
  adding: src/appinventor/ai_youngladesh/GeneratedProject1/Screen1.bky (deflated 84%)
  adding: youngandroidproject/ (stored 0%)
  adding: youngandroidproject/project.properties (deflated 36%)


In [0]:
!mv myapp.zip '$project_name'.aia

In [0]:
!tree myapp

myapp
├── assets
│   ├── bird.wav
│   ├── breakingbad.mp4
│   ├── car.wav
│   ├── lightsaber.wav
│   ├── panini.mp3
│   ├── spiderverse.mp4
│   └── starwars.mp4
├── src
│   └── appinventor
│       └── ai_youngladesh
│           └── GeneratedProject1
│               ├── Screen1.bky
│               └── Screen1.scm
└── youngandroidproject
    └── project.properties

6 directories, 10 files


In [0]:
from google.colab import files
files.download('{}.aia'.format(project_name))

# **Generate synthetic data for training**

### Assets

In [0]:
possible_components = [
 'buttons',
 '<video>',
 '<accelerometer>',
 '<camera>',
 '<text2speech>',
 '<player>'
 ]

In [0]:
output_vocab = [
 '<speak>',
 '<text2speech>',
 '<stop_play>',
 '<text>',
 '</play>',
 '</text>',
 '<play>',
 '<player>',
 '</button>',
 '<end_vis>',
 '<accelerometer>',
 '</shaken>',
 '<open_camera>',
 '<start_code>',
 '<play_video>',
 '<end_code>',
 '<button2>',
 '<video>',
 '</speak>',
 '<new_line>',
 '<button4>',
 '<button>',
 '<shaken>',
 '<camera>',
 '</play_video>',
 '<start_vis>',
 '<button3>',
 '<button1>']

In [0]:
random_sentences = """Leader of the opposition Khaleda Zia led BNP to the parliament session at 3.15pm Thursday after a boycott of 64 days.
Dhaka, Feb 11 (bdnews24.com) â€” Leader of the opposition Khaleda Zia led BNP to the parliament session at 3.15pm Thursday after a boycott of 64 days.
Proceedings began at 3pm, leader of the opposition Khaleda Zia led BNP and coalition MPs into the session at 3.15pm. Speaker Abdul Hamid welcomed them.
Opposition chief whip Zainul Abdin Farroque began by delivering a 20-minute speech, presenting a package of demands to ensure the opposition's stay.
Farroque said the law and order situation had greatly deteriorated in recent times and referred a number of incidents.
Earlier in the day, senior BNP leader Salahuddin Qader Chowdhury said that they would raise issues like law and order and price escalation in parliament.
We would like to raise these burning issues in the parliament ourselves," Chowdhury told bdnews24.com.
70 in Old Dhaka as opposition BNP called a half-day strike protesting the death of Jubo Dal leader Haji Ahmed Hossain.
70 in Old Dhaka as opposition BNP called a half-day strike protesting the death of Jubo Dal leader Haji Ahmed Hossain. BNP also took out a procession in Bangsal at around 10am, witnesses said.
All shops at Bangsal, Alu Bazar, North South Road and Kazi Alauddin Road were closed.
Bangsal Police Station sub-inspector Goutam Roy told bdnews24.com that the strike was being observed peacefully, but police remained alert to avert any untoward incident.
Hossain, ward commissioner of Dhaka City Corporation, was gunned down in Bangshal, Old Dhaka, on Tuesday night.
US Vice President Joe Biden said on Wednesday that al Qaeda was moving in the direction of smaller but "devastatingly frightening" attacks and he viewed the chance of another Sept. 11-style mass assault as unlikely.
In an interview with CNN's "Larry King Live" program, Biden also said the situation in Pakistan worried him more than that of Iraq, Iran and Afghanistan.
"It's a big country," Biden said of Pakistan.
Biden, who was the chairman or ranking member on the Senate Foreign Relations Committee in the decade before becoming vice president, said Iran's nuclear program was a "real concern."
Although U.S. intelligence chiefs testified before a congressional panel recently that new al Qaeda-backed attacks are certain in the coming months, Biden said he thought any attempt would be small-scale.
"If you see what's happening, particularly with al Qaeda and the Arabian Peninsula, they have decided to move in the direction of much more small bore but devastatingly frightening attacks," Biden said.
Umar Farouk Abdulmutallab, 23, is accused of trying to blow up the Amsterdam-to-Detroit flight with a bomb sewn into his underwear.
"The concern relates to somebody like a shoe bomber or the underpants bomber, the Christmas attack or someone just strapping a backpack on them with weapons ... and blowing up," Biden said.
"You're going to see 90,000 American troops come marching home by the end of the summer," Biden said.
"It has impressed me," Biden said.
US Vice President Joe Biden said on Wednesday that al Qaeda was moving in the direction of smaller but "devastatingly frightening" attacks and he viewed the chance of another Sept. 11-style mass assault as unlikely.
In an interview with CNN's "Larry King Live" program, Biden also said the situation in Pakistan worried him more than that of Iraq, Iran and Afghanistan.
"It's a big country," Biden said of Pakistan.
Biden, who was the chairman or ranking member on the Senate Foreign Relations Committee in the decade before becoming vice president, said Iran's nuclear program was a "real concern."
Although US intelligence chiefs testified before a congressional panel recently that new al Qaeda-backed attacks are certain in the coming months, Biden said he thought any attempt would be small-scale.
"If you see what's happening, particularly with al Qaeda and the Arabian Peninsula, they have decided to move in the direction of much more small bore but devastatingly frightening attacks," Biden said.
Umar Farouk Abdulmutallab, 23, is accused of trying to blow up the Amsterdam-to-Detroit flight with a bomb sewn into his underwear.
"The concern relates to somebody like a shoe bomber or the underpants bomber, the Christmas attack or someone just strapping a backpack on them with weapons ... and blowing up," Biden said.
"You're going to see 90,000 American troops come marching home by the end of the summer," Biden said.
"It has impressed me," Biden said.
One who will not is Roman Polanski, whose latest movie "The Ghost Writer" has its world premiere at the event on Friday but who is under house arrest in a Swiss chalet.
Lui Yansheng is not going back to see family members like the others, however, but to seek out his long-lost love and their son who was born after he fled over 50 years earlier.
The Ghost Writer premieres on Friday, and Polanski's arrest in September means media interest in the political thriller starring Ewan McGregor and Pierce Brosnan will be intense.
Polanski reportedly put the finishing touches to the film based on a Robert Harris novel while in jail, and the fictional former British prime minister played by Brosnan is widely understood to be based on Tony Blair.
Also based on a novel, this time by Dennis Lehane, the movie stars DiCaprio as a U.S. marshal who is called to investigate the strange disappearance of a multiple murderer from a maximum security psychiatric clinic.
Kosslick must balance attracting big stars to the red carpet with championing experimental, low-budget cinema from around the globe.
Another major star set to draw crowds in Berlin is Khan, who presents "My Name Is Khan" about an Indian Muslim and his experiences in the United States just after the 9/11 attacks.
One who will not is Roman Polanski, whose latest movie "The Ghost Writer" has its world premiere at the event on Friday but who is under house arrest in a Swiss chalet.
Lui Yansheng is not going back to see family members like the others, however, but to seek out his long-lost love and their son who was born after he fled over 50 years earlier.
The Ghost Writer premieres on Friday, and Polanski's arrest in September means media interest in the political thriller starring Ewan McGregor and Pierce Brosnan will be intense.
Polanski reportedly put the finishing touches to the film based on a Robert Harris novel while in jail, and the fictional former British prime minister played by Brosnan is widely understood to be based on Tony Blair.
Also based on a novel, this time by Dennis Lehane, the movie stars DiCaprio as a U.S. marshal who is called to investigate the strange disappearance of a multiple murderer from a maximum security psychiatric clinic.
Kosslick must balance attracting big stars to the red carpet with championing experimental, low-budget cinema from around the globe.
Another major star set to draw crowds in Berlin is Khan, who presents "My Name Is Khan" about an Indian Muslim and his experiences in the United States just after the 9/11 attacks.
Put into bat, the Tigers rode on a brilliant maiden one-day century by opener Imrul Kayes (138-ball 101) which featured 11 fours.
Tim Southee and Daniel Vettori took three wickets each for the Black Caps.
Secretary-General Ban Ki-moon discussed the arrest of a defeated Sri Lankan opposition candidate with the country's president and expressed concern about events in the Asian nation, the United Nations said on Wednesday.
Tensions have risen in Sri Lanka since former army commander Sarath Fonseka was arrested on Monday by military police on charges of engaging in politics against his president while still in uniform.
Fonseka and Rajapaksa worked together to end a 25-year war against Tamil Tiger separatists last year but soon fell out.
Fonseka lost to Rajapaksa in a January presidential poll and then accused his former commander-in-chief of rigging the vote.
"Ban Ki-moon congratulated President Mahinda Rajapaksa on his outstanding victory," it said in a statement.
The statement, which was posted on the Sri Lankan government's website (www.priu.gov.lk), did not mention Fonseka.
A Hobiganj court on Thursday ordered former home state minister Lutfozzaman Babar into remand for six days for interrogation in connection with the Shah AMS Kibria murder case.
Dhaka, Feb 11 (bdnews24.com) â€” A Hobiganj court on Thursday ordered former home state minister Lutfozzaman Babar into remand for six days for interrogation in connection with the Shah AMS Kibria murder case.
Chief judicial magistrate Rajeeb Kumar Biswas granted the remand prayer by CID's Rafiqul Islam, investigation officer of the case.
Babar was taken to Hobiganj district jail from Dhaka Central Jail on Wednesday and produced before the court Thursday amid tight security.
Earlier in the day, the High Court ordered the police not to torture Babar during his remand in custody.
The bench of justice Nazmun Ara Sultana and justice Md Rais Uddin gave the order following a petition by Babar's wife.
The court in its order asked the police to strictly follow a directive on remand issued by justice Hamidul Haque in 2003.
Babar was remanded Thursday for quizzing over his alleged involvement in the 2005murder of Kibria, a sitting AL MP at the time and former finance minister.
During the hearing at the High Court, attorney general Mahbubey Alam argued for the state while Rafique-ul-Huq and Mahbub Uddin Khokan represented Babar.
Babar was shown arrested in the case on Feb 2, following an order by a judicial magistrate in Habiganj upon an appeal by criminal investigation department.
Public prosecutor Akbar Hossain Jitu earlier said that Babar had 'confessed' his connection with the killing of Kibria, while being interrogated in the August 21 grenade attack case.
CID police super Rafiqul Islam, who is the investigation officer, asked for a ten-day remand for further questioning.
Babar has already been shown arrested in connection with August 21, 2004 grenade attack on an Awami League rally when he was state minister.
The CID police interrogated Babar and two other suspects last month for allegedly suppressing the grenade attack case of 2005 in Habiganj that killed Kibria and four others.
Mahmudur Rahman, acting editor of Bangla-language Dainik Amar Desh, escaped an attack on his car at Tejgaon in the capital on Thursday.
Dhaka, Feb 11 (bdnews24.com) â€“ Mahmudur Rahman, acting editor of Bangla-language Dainik Amar Desh, escaped an attack on his car at Tejgaon in the capital on Thursday.
Unknown assailants attacked the car with a hammer that shattered the rear windshield, Tejgaon zone assistant commissioner of police Mahtabuddin Ahmed told bdnews24.com. "However the editor was unhurt," Ahmed said.
A reporter of the daily, present during the attack, said: "Assailants threw a hammer towards (Rahman's) Pajero jeep.
Rahman was returning to his Uttara residence from the newspaper's Karwan Bazar office at the time of the attack.
Iran's President Mahmoud Ahmadinejad said Israel should be resisted and finished off if it launched military action in the region, state broadcaster IRIB reported on Thursday.
TEHRAN, Feb 11 (bdnews24.com/Reuters) - Iran's President Mahmoud Ahmadinejad said Israel should be resisted and finished off if it launched military action in the region, state broadcaster IRIB reported on Thursday.
Ahmadeinjad's comments were made when the president spoke over the telephone with his Syrian counterpart late on Wednesday.
Lebanese Prime Minister Saad al-Hariri, in an interview broadcast on Wednesday, said Israeli aircraft were making daily incursions into Lebanese air space, creating a very dangerous situation.
Ahmadinejad, who has often predicted the imminent demise of the Jewish state, said Iran would remain on the side of regional nations including Syria, Lebanon and Palestine.
In a statement late last month, Israeli Prime Minister Benjamin Netanyahu said Israel was not planning any imminent attack on Lebanon, from where Hezbollah launched some 4,000 rockets at it during the 34-day war in 2006.
Israeli Foreign Minister Avigdor Lieberman, responding to Syria's accusation last week, has said Damascus would be defeated and Assad would lose power in any future conflict. Netanyahu later reassured Syria that Israel sought peace.
"The message to the people of the area is of course, keep your heads down, stay inside when the operation is going ahead," NATO civilian representative Mark Sedwill said this week.
In Afghanistan, however, NATO commander General Stanley McChrystal's counter-insurgency strategy emphasises seizing population centres and avoiding combat in built-up areas whenever possible.
Party secretary general Ali Ahsan Mohammed Mujahid announced the programme on Thursday at its office at Dhaka's Moghbazar.
Shibir activist Shaheen, 24, of Rajshahi College, died in Chapainawabganj early on Thursday in a gunfight with police.
Police claimed that Shaheen, from Meherchandi in Rajshahi, was an accused in the killing of RU Chhatra League activist Faruque Hossain.
Faruque was killed during Shibir and Chhatra League clashes at the university in the early hours of Tuesday.
Two cases were filed on Wednesday against some 600 Shibir members, including 64 who were named specifically, in connection with the RU violence and Hossain's killing.
When Chinese Foreign Ministry spokesman Ma Zhaoxu was asked that question on Tuesday, however, he avoided directly addressing sanctions, or even using that word.
Speaking in Munich on Friday, Chinese Foreign Minister Yang Jiechi urged more diplomatic efforts. He too did not directly say whether China backed or opposed fresh sanctions.
Nathan Layne, Taiga Uranaka and Kevin Krolicki TOYOTA CITY/DETROIT (bdnews24.com/Reuters) - The problem was mundane but potentially lethal: floor mats were trapping the accelerator pedal.
Last December, Ron Medford, the acting head of the U.S. agency that regulates auto safety, had been quietly dispatched by the Obama administration to deliver a firm message.
But five previous investigations into Toyota opened by NHTSA under the Bush administration had hit a dead end, with no action taken.
"Toyota had the perfect model for the 1980s and 1990s, but its approach now looks outdated," said Stefan Lippert, a business professor at Temple University in Japan.
It is a phrase that Akio Toyoda, grandson of the company's founder, is fond of trotting out.
Across the table was Toyota's top officer in charge of quality, Hiroyuki Yokoyama, and the head of the engineering team that handled consumer complaints, Shinji Miyamoto.
In a company that built its reputation on an almost paranoid obsession with quality, Yokoyama and Miyamoto were the keepers of the flame.
"At that point we weren't expecting the discussions to have any deep meaning because at that point we had already dealt with the floormat issue," Toyota Executive Vice President Shinichi Sasaki said.
NHTSA officials chastised Toyota for "still talking in those terms," Sasaki recalled.
Toyota president Katsuaki Watanabe, who held the top job until June 2009, would tell them to build a car that can go from New York to California on a single tank of gas.
After two weeks largely out of the spotlight, Toyoda called a hasty press conference in Nagoya to apologize.
On Tuesday, the Washington Post ran an editorial comment by Akio Toyoda, a day before the first congressional committee had been scheduled to meet.
CRACKS IN THE ARMOR In October 2007, Dave Champion, director of auto testing for Consumer Reports in Connecticut, was confronted by a shocking result from the magazine's influential survey of subscribers.
"If we believe a vehicle played a significant part in causing damages, we go back to the manufacturer," said spokesman Kip Diggs.
"When you start to see significant claims activity that indicates there may be widespread problems with a product, that's when you go to NHTSA," said Diggs.
To their minds, the goal had always been intended as one of Toyota's audacious stretch targets, like Watanabe's vision of a car that would clean the air.
1," said Yoshi Inaba, who was summoned out of retirement to set right U.S. operations.
Kazuo Akatsuka, 55, saw the generational change first-hand and worried at the signs of change at the Tsutsumi plant where they build the Prius.
They deal with it," AutoNation CEO Mike Jackson said in September, before the first of two massive recalls.
On Tuesday, January 19 Inaba, Toyota U.S. sales chief Jim Lentz and others were summoned to Washington.
When asked to comment that afternoon, Toyota spokesman Brian Lyons said talk of the recall was "an unsubstantiated rumor."
But consumers were not informed that Toyota had found the flaw or developed a fix for the software controlling the Prius brakes until safety engineer Yokoyama told reporters in Tokyo on February 4.
"The damage to the reputation has been done," said Jeff Hess, a professor of marketing at California Polytechnic State University and former auto industry analyst with J.D.
(VIDEO) Nathan Layne, Taiga Uranaka and Kevin Krolicki TOYOTA CITY/DETROIT (bdnews24.com/Reuters) - The problem was mundane but potentially lethal: floor mats were trapping the accelerator pedal.
Last December, Ron Medford, the acting head of the U.S. agency that regulates auto safety, had been quietly dispatched by the Obama administration to deliver a firm message.
But five previous investigations into Toyota opened by NHTSA under the Bush administration had hit a dead end, with no action taken.
"Toyota had the perfect model for the 1980s and 1990s, but its approach now looks outdated," said Stefan Lippert, a business professor at Temple University in Japan.
It is a phrase that Akio Toyoda, grandson of the company's founder, is fond of trotting out.
Across the table was Toyota's top officer in charge of quality, Hiroyuki Yokoyama, and the head of the engineering team that handled consumer complaints, Shinji Miyamoto.
""".split("\n")

In [0]:
button_list = ['button1', 'button2', 'button3', 'button4']
max_num_button = len(button_list)

In [0]:
actuators = [
 '<video>',
 '<camera>',
 '<text2speech>',
 '<player_play>',
 '<player_stop>'
 ]

In [0]:
texts_for_inputs = {
    "button1" : "when button1 pressed , ",
    "button2" : "when button2 pressed , ",
    "button3" : "when button3 pressed , ",
    "button4" : "when button4 pressed , ",
    "accelerometer" : "when phone is shaken , "
}
texts_for_outputs = {
  "<video>": "play video {} . ",
  "<camera>": "open the camera . ",
  "<text2speech>" : "say , <speech> . ",
  "<player_play>": "play {} . ",
  "<player_stop>": "stop player . "
}

In [0]:
sar_gen_dict = {
    "button1" : "<button1> {} </button> <new_line> ",
    "button2" : "<button2> {} </button> <new_line> ",
    "button3" : "<button3> {} </button> <new_line> ",
    "button4" : "<button4> {} </button> <new_line> ",
    "accelerometer" : "<shaken> {} </shaken> <new_line> ",
    '<video>': " <play_video> {} </play_video> ",
    '<camera>': " <open_camera> ",
    '<text2speech>' : " <speak> <text> <speech> </text> </speak> ",
    '<player_play>':  " <play> {} </play> ",
    '<player_stop>': " <stop_play> "
}

In [0]:
audio_files = ['bird.wav','car.wav','panini.mp3','lightsaber.wav']
video_files = ['starwars.mp4','breakingbad.mp4','spiderverse.mp4']

### Text and Code generation

In [0]:
num_button = random.randrange(0,5)

In [0]:
take_accelerometer = random.randrange(0,2)

In [0]:
#num = random.randrange(0,len(random_sentences))
#random_sentences[num]

In [0]:
accepted_inputs = []
for i in range(num_button):
  accepted_inputs.append(button_list[i])
if num_button == 0:
  accepted_inputs.append('accelerometer')
  take_accelerometer = 1
elif take_accelerometer:
  accepted_inputs.append('accelerometer')
accepted_inputs

['button1', 'button2', 'button3', 'button4', 'accelerometer']

In [0]:
accepted_outputs = []
for i in range(len(accepted_inputs)):
  num = random.randrange(0,len(actuators))
  accepted_outputs.append(actuators[num])
accepted_outputs

['<player_stop>', '<text2speech>', '<video>', '<player_play>', '<video>']

In [0]:
text = "create an app with "

In [0]:
if num_button ==1:
  text += str(num_button) + " button "
elif num_button != 0:
  text += str(num_button) + " buttons "
if num_button != 0 and take_accelerometer:
  text += " and an accelerometer sensor "
elif take_accelerometer:
  text += " an accelerometer sensor "
text += ". "

In [0]:
text

'create an app with 4 buttons  and an accelerometer sensor . '

In [0]:
input_stream = "<start_vis> {} <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> ".format(' '.join(['<button>' for i in range(num_button)]))
for (x,y) in zip(accepted_inputs, accepted_outputs):
  output_code = sar_gen_dict[y]
  if y == '<video>':
    num = random.randrange(0,len(video_files))
    video_file = video_files[num]
    output_text = texts_for_outputs[y].format(video_file)
    output_code = output_code.format(video_file)
  elif y == '<player_play>':
    num = random.randrange(0,len(audio_files))
    audio_file = audio_files[num]
    output_text = texts_for_outputs[y].format(audio_file)
    output_code = output_code.format(audio_file)
  else:
    output_text = texts_for_outputs[y]

  code_text = sar_gen_dict[x].format(output_code)
  text += texts_for_inputs[x]
  text += output_text
  input_stream += code_text

input_stream += "<end_code>"


In [0]:
input_stream

'<start_vis> <button> <button> <button> <button> <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> <button1>  <stop_play>  </button> <new_line> <button2>  <speak> <text> <speech> </text> </speak>  </button> <new_line> <button3>  <play_video> breakingbad.mp4 </play_video>  </button> <new_line> <button4>  <play> panini.mp3 </play>  </button> <new_line> <shaken>  <play_video> starwars.mp4 </play_video>  </shaken> <new_line> <end_code>'

In [0]:
text

'create an app with 4 buttons  and an accelerometer sensor . when button1 pressed , stop player . when button2 pressed , say , <speech> . when button3 pressed , play video breakingbad.mp4 . when button4 pressed , play panini.mp3 . when phone is shaken , play video starwars.mp4 . '

In [0]:
"i have dollars".find('dolldar')

-1

In [0]:
def num_button_to_token(text):
  if text.find('button') == -1:
    return text
  list_of_numbers = [" no ", " one ", " two ", " three ", " four "]
  for i in range(len(list_of_numbers)):
    text = text.replace(list_of_numbers[i], " "+str(i)+" ")
  num_button_str = re.findall("[0-9]+ buttons?", text)[0]
  num_button_found = int(num_button_str.split()[0])
  num_button_changed_string = ""
  for i in range(min(num_button_found, max_num_button)):
    num_button_changed_string += button_list[i] + " "
  return text.replace(num_button_str ,num_button_changed_string)

In [0]:
num_button_to_token(text)

'create an app with button1 button2 button3 button4   and an accelerometer sensor . when button1 pressed , play video starwars.mp4 . when button2 pressed , play bird.wav . when button3 pressed , stop player . when button4 pressed , play video breakingbad.mp4 . when phone is shaken , play video starwars.mp4 . '

In [0]:
def extract_speech(text):
  speech = re.findall("\".+\"", text)
  if speech:
    speech = speech[0][1:-1]
  text2 = re.sub("\".+\"", "<speech>", text)
  return text2, speech

In [0]:
def make_tokenizable(real_text):
  special_tokens = [".", ",", '"', "'"]
  for x in special_tokens:
    real_text = real_text.replace(x, " "+x+" ")
  return real_text

### One Function for all Generation

In [0]:
def generate_text_code_pair():
  num_button = random.randrange(0,5)
  take_accelerometer = random.randrange(0,2)
  accepted_inputs = []
  for i in range(num_button):
    accepted_inputs.append(button_list[i])
  if num_button == 0:
    accepted_inputs.append('accelerometer')
    take_accelerometer = 1
  elif take_accelerometer:
    accepted_inputs.append('accelerometer')
  accepted_outputs = []
  for i in range(len(accepted_inputs)):
    num = random.randrange(0,len(actuators))
    accepted_outputs.append(actuators[num])
  text = "create an app with "
  if num_button ==1:
    text += str(num_button) + " button "
  elif num_button != 0:
    text += str(num_button) + " buttons "
  if num_button != 0 and take_accelerometer:
    text += " and an accelerometer sensor "
  elif take_accelerometer:
    text += " an accelerometer sensor "
  text += ". "

  input_stream = "<start_vis> {} <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> ".format(' '.join(['<button>' for i in range(num_button)]))
  for (x,y) in zip(accepted_inputs, accepted_outputs):
    output_code = sar_gen_dict[y]
    if y == '<video>':
      num = random.randrange(0,len(video_files))
      video_file = video_files[num]
      output_text = texts_for_outputs[y].format(video_file)
      output_code = output_code.format(video_file)
    elif y == '<player_play>':
      num = random.randrange(0,len(audio_files))
      audio_file = audio_files[num]
      output_text = texts_for_outputs[y].format(audio_file)
      output_code = output_code.format(audio_file)
    else:
      output_text = texts_for_outputs[y]

    code_text = sar_gen_dict[x].format(output_code)
    text += texts_for_inputs[x]
    text += output_text
    input_stream += code_text
  input_stream += "<end_code>"

  return text, input_stream

In [0]:
texts = []
sars = []
for i in range(500):
  text, sar = generate_text_code_pair()
  text = num_button_to_token(text)
  texts.append(text)
  sars.append(sar)

In [38]:
texts

['create an app with button1 button2  . when button1 pressed , play video breakingbad.mp4 . when button2 pressed , stop player . ',
 'create an app with  an accelerometer sensor . when phone is shaken , open the camera . ',
 'create an app with button1 button2  . when button1 pressed , open the camera . when button2 pressed , play video starwars.mp4 . ',
 'create an app with button1 button2 button3 button4  . when button1 pressed , open the camera . when button2 pressed , stop player . when button3 pressed , open the camera . when button4 pressed , open the camera . ',
 'create an app with  an accelerometer sensor . when phone is shaken , play video spiderverse.mp4 . ',
 'create an app with button1  . when button1 pressed , stop player . ',
 'create an app with button1 button2   and an accelerometer sensor . when button1 pressed , play video starwars.mp4 . when button2 pressed , open the camera . when phone is shaken , say , <speech> . ',
 'create an app with button1 button2  . when bu

In [39]:
sars

['<start_vis> <button> <button> <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> <button1>  <play_video> breakingbad.mp4 </play_video>  </button> <new_line> <button2>  <stop_play>  </button> <new_line> <end_code>',
 '<start_vis>  <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> <shaken>  <open_camera>  </shaken> <new_line> <end_code>',
 '<start_vis> <button> <button> <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> <button1>  <open_camera>  </button> <new_line> <button2>  <play_video> starwars.mp4 </play_video>  </button> <new_line> <end_code>',
 '<start_vis> <button> <button> <button> <button> <accelerometer> <video> <camera> <text2speech> <player> <end_vis> <start_code> <button1>  <open_camera>  </button> <new_line> <button2>  <stop_play>  </button> <new_line> <button3>  <open_camera>  </button> <new_line> <button4>  <open_camera>  </button> <new_line> <end_code>',
 '<start_vis>  <accelerometer>

In [0]:
df = pd.DataFrame({'input_text':texts, 'target_sar': sars})

In [0]:
df.to_csv("text2app_data_1000.csv", index=False)

In [0]:
from google.colab import files

files.download('text2app_data_1000.csv')